## Imports

In [1]:
import logging
import random
from functools import reduce

logging.getLogger().setLevel(logging.INFO)

## Define the class Nim

In [2]:
class Nim:
    def __init__(self, num_rows: int, k: int = None) -> None:
        self._rows = [i*2+1 for i in range(num_rows)]
        self._k = k

    def nimming_remove(self, row: int, num_objects: int) -> None:
        assert self._rows[row] >= num_objects
        assert self._k is None or num_objects < self._k
        self._rows[row] -= num_objects
    
    def nimming_add(self, row: int, num_objects: int) -> None:
        self._rows[row] += num_objects

    def goal(self) -> bool:
        # Check if someone has won
        return sum(self._rows) == 0

## Random player

In [3]:
class RandomPlayer:
    def __init__(self) -> None:
        self._moves = []
        self._nMoves = 0
    
    def play(self, nim: Nim) -> None:
        # Chose a random row and a random number of pieces
        x = random.randint(0, len(nim._rows)-1)
        while nim._rows[x] == 0:
            x = random.randint(0, len(nim._rows)-1)
        y = random.randint(1, nim._rows[x])
        # Update the attributes
        self._nMoves += 1
        self._moves.append(f"{y} items from row {x}")
        nim.nimming_remove(x, y)
        
    def printSolution(self) -> None:
        logging.info(f" Random player won, moves have been: {self._moves}; total: {self._nMoves}")

## Task3.1: An agent using fixed rules based on nim-sum (i.e., an expert system)

In [46]:
def verify_state(nrow: int, num_objects: int, nim: Nim) -> bool:
    # Verify of the new game state is useful for the player with the nim sum rule
    xor = reduce(lambda a, b: a ^ b, nim._rows)
    if xor == 0:
        return False
    else:
        return True

### Expert System

In [47]:
class ExpertSystem:
    def __init__(self) -> None:
        self._nMoves = 0
        self._moves = []

    def printSolution(self) -> None:
        logging.info(f" Expert system won, moves have been: {self._moves}; total: {self._nMoves}")
    
    def play(self, nim: Nim) -> None:
        x = 0
        y = 0
        valid = False
        flag = True
        not_found = False
        x_already_taken = []
        x_can_be_taken = [n for n,row in enumerate(nim._rows) if row!=0]
        # Chose a random row and a random number of pieces
        while flag:
            x = random.randint(0, len(nim._rows)-1)
            # If I have already tried the row x
            while x in x_already_taken or x not in x_can_be_taken:
                if len(x_already_taken) == len(x_can_be_taken):
                    # All rows have already been tried
                    flag = False
                    not_found = True
                    break
                x = random.randint(0, len(nim._rows)-1)
            x_already_taken.append(x)
            y_already_taken = []
            while flag == True and valid == False and len(y_already_taken) < nim._rows[x]:
                y = random.randint(1, nim._rows[x])
                if y not in y_already_taken:
                    # If the deleting of y objects has not been already tried
                    y_already_taken.append(y)
                    nim.nimming_remove(x, y)
                    valid = verify_state(x, y, nim)
                    if valid == True:
                        # Found a good move
                        flag = False
                    else:
                        # Restore the game as before if the solution is useless for the player (search for another)
                        nim.nimming_add(x, y)

        if not_found:
            # If no valid moves were found, do a random move
            x = random.randint(0, len(nim._rows)-1)
            while nim._rows[x] == 0:
                x = random.randint(0, len(nim._rows)-1)
            y = random.randint(1, nim._rows[x])
            nim.nimming_remove(x, y)
            
        self._nMoves += 1
        self._moves.append(f"{y} items from row {x}")

### Game: Random vs Expert

In [ ]:
nim = Nim(7)
player1 = RandomPlayer()
player2 = ExpertSystem()
win = random.choice([1, 2])
print("Start game: ", nim._rows)
while nim.goal() == False:
    if win == 1:
        player1.play(nim)
        print("Game after Random player played: ", nim._rows)
        win = 2
    else:
        player2.play(nim)
        print("Game after Expert system played: ", nim._rows)
        win = 1
if win == 1:
    player1.printSolution()
else:
    player2.printSolution()

## Task3.2: An agent using evolved rules

## Task3.3: An agent using minmax

### MinMax System

In [8]:
class MinMaxSystem:
    def __init__(self) -> None:
        self._nmoves = 0
        self._moves = []

    def printSolution(self) -> None:
        logging.info(f" MinMax system won, moves have been: {self._moves}; total: {self._nmoves}")

    def minmax(self, nim: Nim, player: bool):
        possible = [(r, o) for r, c in enumerate(nim._rows) for o in range(1, c + 1)]
        # True player is the MinMax system, False player is the other player
        if not possible:
            if player:
                # Win -> the MinMax system arrives with the table empty
                return (None, 1, None)
            else:
                # No win -> the other player arrives with the table empty
                return (None, -1, None)
        evaluations = list()
        for ply in possible:
            nim.nimming_remove(ply[0], ply[1])
            # Recursive call
            _, val = self.minmax(nim, not player)
            evaluations.append((ply, val))
            # Restore the previous situation for another evaluation
            nim.nimming_add(ply[0], ply[1])
        return max(evaluations, key=lambda k: k[1])

    def play(self, nim: Nim):
        best_ply, _ = self.minmax(nim, player=True)
        nim.nimming_remove(best_ply[0], best_ply[1])
        self._nmoves += 1
        self._moves.append(f"{best_ply[1]} items from row {best_ply[0]}")

### Game: Random vs MinMax

In [9]:
nim = Nim(4)
player1 = RandomPlayer()
player2 = MinMaxSystem()
win = random.choice([1])
print("Start game: ", nim._rows)
while nim.goal() == False:
    if win == 1:
        player1.play(nim)
        print("Game after Random player played: ", nim._rows)
        win = 2
    else:
        player2.play(nim)
        print("Game after MinMax system played: ", nim._rows)
        win = 1
if win == 1:
    player1.printSolution()
else:
    player2.printSolution()

Start game:  [1, 3, 5, 7]
Game after Random player played:  [1, 2, 5, 7]


KeyboardInterrupt: 

## Task3.4: An agent using reinforcement learning